---
title: Calculating temperature anomalies
short_title: Temperature anomalies
---

This notebook shows how to calculate temperature anomalies for districts in Sierra Leone based on data from [ERA5-Land](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means). Temperature anomalies represent the deviation from a baseline period (climate normal), and are essential for understanding climate trends and variations.

A temperature anomaly is calculated by subtracting the long-term average (climate normal) from the observed temperature. Positive anomalies indicate warmer than normal conditions, while negative anomalies indicate cooler than normal conditions.

We will use [earthkit](https://ecmwf.github.io/earthkit-website/), [xarray](https://xarray.dev/) and [pandas](https://pandas.pydata.org/) to perform the analysis.

In [ ]:
import geopandas as gpd
import earthkit.data
from earthkit import transforms
import matplotlib.pyplot as plt

## Loading climate data

Use [earthkit.data](https://earthkit-data.readthedocs.io) to load a NetCDF file containing monthly temperature values since 1990:

In [ ]:
file = "../data/era5-land-monthly-temp-precip-1990-2025-sierra-leone.nc"
data = earthkit.data.from_source("file", file)

This file was downloaded from the "[ERA5-Land monthly averaged data from 1950 to present](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means)" dataset. See [this tutorial](../getting-data/climate-data-store.ipynb) for how to download data from the Climate Data Store.

We convert the data to an [xarray dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html):

In [ ]:
dataset = data.to_xarray()
dataset

The dataset covers Sierra Leone and contains temperature data (`t2m`: Temperature 2m above surface). The spatial resolution for this gridded dataset is approximately 9x9 km, and the temporal resolution is monthly values.

We can drop the variables that we don't need in our analysis:

In [ ]:
dataset_clean = dataset.drop_vars(['number', 'expver'])

## Calculate climate normals (baseline)

To calculate temperature anomalies, we first need to establish a baseline. We'll use the 30-year reference period (1991-2020) as recommended by the World Meteorological Organization (WMO) to calculate [climate normals](../glossary.md).

In [ ]:
dataset_baseline = dataset_clean.sel(valid_time=slice('1991-01-01', '2020-12-01'))

Extract the temperature variable and convert from Kelvin to Celsius:

In [ ]:
temp_baseline = dataset_baseline['t2m'] - 273.15

Calculate the climate normal by grouping by calendar month and computing the mean across all years:

In [ ]:
temp_normal = temp_baseline.groupby('valid_time.month').mean()
temp_normal

## Calculate temperature anomalies

Now we'll calculate temperature anomalies for all available years in our dataset. First, extract the temperature for all years and convert to Celsius:

In [ ]:
temp_all = dataset_clean['t2m'] - 273.15

Calculate the anomaly by subtracting the climate normal from the observed temperature. We group by month to ensure we're comparing each month to its corresponding baseline month:

In [ ]:
temp_anomaly = temp_all.groupby('valid_time.month') - temp_normal
temp_anomaly

## Analyzing recent anomalies

Let's focus on recent years (2021-2025) to see how temperatures have deviated from the baseline:

In [ ]:
temp_anomaly_recent = temp_anomaly.sel(valid_time=slice('2021-01-01', '2025-12-01'))

Calculate the spatial mean (average across all grid points) to get a single anomaly value for each time step:

In [ ]:
anomaly_timeseries = temp_anomaly_recent.mean(dim=['latitude', 'longitude'])

## Visualizing temperature anomalies

Create a time series plot showing how temperatures have varied from the baseline:

In [ ]:
plt.figure(figsize=(12, 6))
anomaly_timeseries.plot()
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
plt.title('Temperature Anomalies for Sierra Leone (2021-2025)')
plt.ylabel('Temperature Anomaly (°C)')
plt.xlabel('Time')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

The plot shows the monthly temperature anomalies. Values above zero indicate months that were warmer than the baseline period, while values below zero indicate cooler months.

## Calculate annual mean anomalies

To see the overall trend, we can calculate the annual mean anomaly:

In [ ]:
annual_anomaly = temp_anomaly_recent.resample(valid_time='1Y').mean()
annual_mean = annual_anomaly.mean(dim=['latitude', 'longitude'])
annual_mean

Visualize the annual anomalies:

In [ ]:
plt.figure(figsize=(10, 6))
annual_mean.plot.bar()
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
plt.title('Annual Mean Temperature Anomalies for Sierra Leone (2021-2025)')
plt.ylabel('Temperature Anomaly (°C)')
plt.xlabel('Year')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Temperature anomalies for districts

Finally, let's calculate temperature anomalies for each district in Sierra Leone. First, load the districts from a GeoJSON file [downloaded from DHIS2 Maps](../org-units/download-maps-app.md):

In [ ]:
districts = gpd.read_file('../data/sierra-leone-districts.geojson')
districts.head()

Use [earthkit.transforms](https://earthkit-transforms.readthedocs.io) to aggregate the anomaly data for each district. We'll calculate the mean anomaly for 2024:

In [ ]:
# Select 2024 data
temp_anomaly_2024 = temp_anomaly.sel(valid_time=slice('2024-01-01', '2024-12-01'))

# Calculate annual mean for 2024
temp_anomaly_2024_annual = temp_anomaly_2024.mean(dim='valid_time')

In [ ]:
# Aggregate by district
result = transforms.aggregate.vector_stats(
    temp_anomaly_2024_annual,
    districts.geometry,
    index=districts.name,
    statistics=["mean"]
)
result

Merge the results with the district geodataframe to create a map:

In [ ]:
districts_anomaly = districts.merge(result, left_on='name', right_index=True)

Create a choropleth map showing the spatial distribution of temperature anomalies:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
districts_anomaly.plot(
    column='mean',
    ax=ax,
    legend=True,
    cmap='RdBu_r',
    edgecolor='black',
    linewidth=0.5,
    legend_kwds={'label': 'Temperature Anomaly (°C)', 'orientation': 'horizontal'}
)
ax.set_title('Mean Temperature Anomaly by District - Sierra Leone (2024)', fontsize=14, fontweight='bold')
ax.set_axis_off()
plt.tight_layout()
plt.show()

## Summary

In this notebook, we:
1. Loaded ERA5-Land monthly temperature data
2. Calculated climate normals (baseline) for the period 1991-2020
3. Computed temperature anomalies by comparing observed temperatures to the baseline
4. Visualized anomalies over time with time series plots
5. Calculated annual mean anomalies
6. Aggregated anomalies by district and created a spatial visualization

Temperature anomalies are valuable for climate monitoring and can help identify warming or cooling trends relative to historical averages.